### TCC Comparação de desempenho de predição temporal, entre modelo Linear, MLPs, CNNs e LSTMs

#### Esse Notebook foi desenvolvido exclusivamente por:
- #### Victor Mario Bertini RA:194761

## 1° Etapa:
##### Coleta de dados da fonte a ser utilziada

Os dados a serem utilziados para o projeto serão extraídos da fonte https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html

A coleta de dados será feita inicialmente através da RESTFUL API fornecida pela fonte.

Os dados serão carregados e salvos em formatos de CSV e armazenados na pasta _data_

A fonte inclui informações elétricas relevantes de geração, consumo, transmissão, entre outros. Todos os dados são referentes a países Europeus

O Ponto principal com essa estrutura de dados será contruir, treinar e testar diferentes modelos de redes neurais para executar previsões temporais de multiplas variáveis.

Para esse Notebook serão carregados multiplos dados de múltiplos países europeus referentes apenas a previsão de _CONSUMO_ desses países

Essa primeira etapa será incialmente dividida em 3 Capítulos:

- Coleta e save Bruta de dados
- Separação em 3 estruturas para treinamento:

1. MLPs
2. CNNs
3. LSTMs

- Tratamento dos dataframes treinados 

## 1° Etapa - Capítulo 1: Coleta de dados Brutos


In [ ]:
##Imports necessários para GET request na API

import requests
import pandas

# Definindo a URL base e os parâmetros da requisição
url = "https://web-api.tp.entsoe.eu/api"
params = {
    "documentType": "A65",
    "processType": "A16",
    "outBiddingZone_Domain": "10YCZ-CEPS-----N",
    "periodStart": "202303030000", # Data de Início
    "periodEnd": "202303060000" # Data de Termino
}

# Fazendo a requisição GET para a API
response = requests.get(url, params=params)

# Verificando se a requisição foi bem sucedida
if response.status_code == 200:
    # Os dados retornam em XML, então precisamos fazer o parsing
    import xml.etree.ElementTree as ET
    root = ET.fromstring(response.content)
    
    # Exemplo de extração de dados do XML (ajuste conforme a estrutura do XML retornado)
    data = []
    for timeseries in root.findall('.//{*}TimeSeries'):
        for period in timeseries.findall('.//{*}Period'):
            for point in period.findall('.//{*}Point'):
                position = point.find('{*}position').text
                quantity = point.find('{*}quantity').text
                data.append({'position': position, 'quantity': quantity})
    
    # Criando o DataFrame
    df = pandas.DataFrame(data)
    
    # Salvando o DataFrame em CSV na pasta data
    df.to_csv('data/consumo_pais.csv', index=False)
else:
    print(f"Erro na requisição: {response.status_code}")